In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec, VectorAssembler, Normalizer

In [2]:
spark = SparkSession.builder \
    .appName("ClusteringWord2Vec") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/12/01 16:41:59 WARN Utils: Your hostname, matt resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/01 16:41:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/01 16:42:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/01 16:42:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/01 16:42:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
df_spark = spark.read.parquet("../data/dblp-v10-processado.parquet")

In [5]:
# 1. Word2Vec para 'title'
word2vec_title = Word2Vec(vectorSize=100, inputCol="title", outputCol="title_word2vec")
model_title = word2vec_title.fit(df_spark)
df_spark = model_title.transform(df_spark)

24/12/01 16:42:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/01 16:42:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [6]:
# 2. Word2Vec para 'abstract'
word2vec_abstract = Word2Vec(vectorSize=100, inputCol="abstract", outputCol="abstract_word2vec")
model_abstract = word2vec_abstract.fit(df_spark)
df_spark = model_abstract.transform(df_spark)

In [7]:
# 3. Combinação dos vetores
assembler = VectorAssembler(inputCols=["title_word2vec", "abstract_word2vec"], outputCol="features")
df_spark = assembler.transform(df_spark)

In [8]:
# 4. Normalização
normalizer = Normalizer(inputCol="features", outputCol="norm_features")
df_spark = normalizer.transform(df_spark)

In [9]:
df_spark.columns

['abstract',
 'title',
 'title_word2vec',
 'abstract_word2vec',
 'features',
 'norm_features']